In [1]:
import pandas as pd
import numpy as np
from datetime import date, datetime, timedelta


In [2]:
# Re-importing the test.csv dataset using the correct date column name

Path = '/Users/lesegomatojane/Documents/MIT807/Muliti-transformer/MultiTransformer-master/MultiTransformer/Train/assets/Drop001/5_MTL_GARCH_AAPL.csv'
GARCH_mpt = pd.read_csv(Path, index_col='Date_Forecast', parse_dates=True)
GARCH_mpt 

,h1,h2,h3,h4,TrueSD
Date_Forecast,,,,,
2007-12-14,0.036723,0.042510,0.035440,0.036478,0.021336
2007-12-28,0.034850,0.068866,0.043401,0.041632,0.078664
2008-01-04,0.042266,0.040022,0.041769,0.037158,0.103452
2008-01-04,0.032792,0.034442,0.033476,0.036393,0.097676
2008-01-18,0.046529,0.032525,0.041137,0.038367,0.023060
...,...,...,...,...,...
2015-11-20,0.041264,0.036400,0.052939,0.051809,0.014581
2015-11-27,0.036896,0.031423,0.026599,0.017492,0.026239
2015-12-04,0.028170,0.018024,0.026134,0.032488,0.029364


In [3]:
# Load the updated data
df_updated = GARCH_mpt.copy()
df_updated 
# Calculate MAE for each horizon
mae_h1_updated = np.mean(np.abs(df_updated['h1'] - df_updated['TrueSD']))
mae_h2_updated = np.mean(np.abs(df_updated['h2'] - df_updated['TrueSD'].shift(1))[1:])
mae_h3_updated = np.mean(np.abs(df_updated['h3'] - df_updated['TrueSD'].shift(2))[2:])
mae_h4_updated = np.mean(np.abs(df_updated['h4'] - df_updated['TrueSD'].shift(3))[3:])

# Calculate MAE of the naive forecast (using last known value)
mae_naive_updated = np.mean(np.abs(df_updated['TrueSD'].shift(1) - df_updated['TrueSD'])[1:])

# Compute MASE for each horizon
mase_h1_updated = mae_h1_updated / mae_naive_updated
mase_h2_updated = mae_h2_updated / mae_naive_updated
mase_h3_updated = mae_h3_updated / mae_naive_updated
mase_h4_updated = mae_h4_updated / mae_naive_updated

mase_h1_updated, mase_h2_updated, mase_h3_updated, mase_h4_updated

(0.9826296843493042, 0.9849083310976302, 0.9726282506431666, 0.933637086945277)

# Aggregate Monthly

In [4]:
# Convert the index to datetime format

# Resample to monthly frequency
df_monthly = df_updated.resample('M').last()

df_monthly.head()

,h1,h2,h3,h4,TrueSD
Date_Forecast,,,,,
2007-12-31,0.034850,0.068866,0.043401,0.041632,0.078664
2008-01-31,0.033059,0.045750,0.038480,0.047257,0.051929
2008-02-29,0.043157,0.049862,0.042358,0.053783,0.082800
2008-03-31,0.039952,0.046065,0.055690,0.046123,0.044654
2008-04-30,0.058646,0.050566,0.046496,0.046131,0.042379


In [5]:
# Adjust the compute_monthly_mase function to use a constant naive forecast for each horizon
def compute_direct_monthly_mase(row, forecast_col, actual_col='TrueSD', naive_forecast_col='naive_forecast'):
    """
    Compute MASE for a given month using a specific forecast column for direct multi-step forecasting.
    """
    mae = np.abs(row[forecast_col] - row[actual_col])
    mae_naive = np.abs(row[naive_forecast_col] - row[actual_col])
    return mae / mae_naive

# Add a column for naive forecast (previous month's TrueSD value)
df_monthly['naive_forecast'] = df_monthly['TrueSD'].shift(1)

# Compute monthly MASE for each forecast horizon using the adjusted function
mase_h1_direct_monthly = df_monthly.apply(compute_direct_monthly_mase, forecast_col='h1', axis=1)
mase_h2_direct_monthly = df_monthly.apply(compute_direct_monthly_mase, forecast_col='h2', axis=1)
mase_h3_direct_monthly = df_monthly.apply(compute_direct_monthly_mase, forecast_col='h3', axis=1)
mase_h4_direct_monthly = df_monthly.apply(compute_direct_monthly_mase, forecast_col='h4', axis=1)

# Combine into a DataFrame for better visualization
mase_direct_monthly_df = pd.DataFrame({
    'MASE_h1': mase_h1_direct_monthly,
    'MASE_h2': mase_h2_direct_monthly,
    'MASE_h3': mase_h3_direct_monthly,
    'MASE_h4': mase_h4_direct_monthly
})

mase_direct_monthly_df


,MASE_h1,MASE_h2,MASE_h3,MASE_h4
Date_Forecast,,,,
2007-12-31,NaN,NaN,NaN,NaN
2008-01-31,0.705842,0.231137,0.503069,0.174776
2008-02-29,1.284167,1.066958,1.310046,0.939957
2008-03-31,0.123261,0.036980,0.289320,0.038500
2008-04-30,7.150953,3.598849,1.809931,1.649203
...,...,...,...,...
2015-08-31,1.933658,1.602099,2.374791,1.233859
2015-09-30,0.307117,0.058341,0.135992,0.042607
2015-10-31,0.630163,0.627737,0.928586,0.895481


In [1]:
np.mean(mase_direct_monthly_df)

NameError: name 'np' is not defined

In [8]:
mase_direct_monthly_df.to_csv('AAPL_MASE.csv')

In [15]:
mase_direct_monthly_df

,MASE_h1,MASE_h2,MASE_h3,MASE_h4
Date_Forecast,,,,
2007-12-31,NaN,NaN,NaN,NaN
2008-01-31,0.705842,0.231137,0.503069,0.174776
2008-02-29,1.284167,1.066958,1.310046,0.939957
2008-03-31,0.123261,0.036980,0.289320,0.038500
2008-04-30,7.150953,3.598849,1.809931,1.649203
...,...,...,...,...
2015-08-31,1.933658,1.602099,2.374791,1.233859
2015-09-30,0.307117,0.058341,0.135992,0.042607
2015-10-31,0.630163,0.627737,0.928586,0.895481


In [23]:
import plotly.graph_objects as go

# Create the interactive line plot using Plotly
fig = go.Figure()

# Add lines for each forecast horizon with different markers
fig.add_trace(go.Scatter(x=mase_direct_monthly_df.index, y=mase_direct_monthly_df['MASE_h1'], mode='lines+markers', name='MASE_h1', marker=dict(symbol='circle')))
fig.add_trace(go.Scatter(x=mase_direct_monthly_df.index, y=mase_direct_monthly_df['MASE_h2'], mode='lines+markers', name='MASE_h2', marker=dict(symbol='square')))
fig.add_trace(go.Scatter(x=mase_direct_monthly_df.index, y=mase_direct_monthly_df['MASE_h3'], mode='lines+markers', name='MASE_h3', marker=dict(symbol='diamond')))
fig.add_trace(go.Scatter(x=mase_direct_monthly_df.index, y=mase_direct_monthly_df['MASE_h4'], mode='lines+markers', name='MASE_h4', marker=dict(symbol='cross')))

# Add a horizontal line at MASE=1 for reference
fig.add_shape(
    type='line',
    line=dict(dash='dash', color='white'),
    x0=mase_direct_monthly_df.index[0],
    x1=mase_direct_monthly_df.index[-1],
    y0=1,
    y1=1
)

# Update the layout for transparent background and other stylistic choices
fig.update_layout(
    title='MASE values for Different Forecast Horizons over Time',
    xaxis_title='Date',
    yaxis_title='MASE',
    template="none",
    yaxis=dict(type='log', gridcolor='gray', tickcolor='gray', tickfont=dict(color='white')),
    xaxis=dict(gridcolor='gray', tickcolor='gray', tickfont=dict(color='white')),
    plot_bgcolor='rgba(0,0,0,0)',     # Transparent background for the plot area
    paper_bgcolor='rgba(0,0,0,0)',   # Transparent background for the whole figure
    font=dict(color='white'),
    legend=dict(font=dict(color='white'))
)

fig.show()


In [ ]:
# Function to convert DataFrame to a format suitable for pgfplots
def dataframe_to_pgfplots(df):
    output_str = []
    for index, row in df.iterrows():
        row_str = ' '.join(map(str, row.values))
        output_str.append(row_str)
    return '\n'.join(output_str)

# Extract data for pgfplots
pgf_data = mase_direct_monthly_df.drop(columns="Date_Forecast")
pgf_data_str = dataframe_to_pgfplots(pgf_data)

# Construct LaTeX code with pgfplots
latex_code = r"""
\documentclass{standalone}
\usepackage{pgfplots}
\pgfplotsset{compat=1.17}

\begin{document}
\begin{tikzpicture}
\begin{axis}[
    width=15cm, height=8cm,
    title={MASE values for different forecast horizons},
    xlabel={Forecast Date},
    ylabel={MASE Value},
    date coordinates in=x,
    xtick=data,
    xticklabel style={rotate=45,anchor=north east},
    xticklabels from table={data.dat}{Date_Forecast},
    grid=major,
    legend pos=north west,
]

\addplot table[x=Date_Forecast, y=MASE_h1] {data.dat};
\addlegendentry{MASE\_h1}

\addplot table[x=Date_Forecast, y=MASE_h2] {data.dat};
\addlegendentry{MASE\_h2}

\addplot table[x=Date_Forecast, y=MASE_h3] {data.dat};
\addlegendentry{MASE\_h3}

\addplot table[x=Date_Forecast, y=MASE_h4] {data.dat};
\addlegendentry{MASE\_h4}

\end{axis}
\end{tikzpicture}
\end{document}
"""

latex_code
